# Computer Vision Homework 3: Big vs Small Models

## Brief

Due date: Nov 16, 2022

Required files: `homework-3.ipynb`, `report.pdf`

To download the jupyter notebook from colab, you can refer to the colab tutorial we gave.


## Codes for Problem 1 and Problem 2

### Import Packages

In [ ]:
import glob
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from PIL import Image
from torch.utils.data import DataLoader, Dataset, RandomSampler
from torchvision import transforms, models, datasets
from tqdm import tqdm

%matplotlib inline

### Check GPU Environment

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

Using cuda device


In [ ]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-ad45f209-d4e8-9f75-e62a-a67b2b849d48)


### Set the Seed to Reproduce the Result

In [ ]:
def set_all_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
set_all_seed(123)

### Create Dataset and Dataloader

In [ ]:
batch_size = 256

train_transform = transforms.Compose([
    transforms.Pad(4, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=train_transform)
valid_dataset = datasets.CIFAR10(root='data', train=False, download=True, transform=test_transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

sixteenth_train_sampler = RandomSampler(train_dataset, num_samples=len(train_dataset)//16)
half_train_sampler = RandomSampler(train_dataset, num_samples=len(train_dataset)//2)

sixteenth_train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sixteenth_train_sampler)
half_train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=half_train_sampler)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


### Load Models

In [ ]:
# HINT: Remember to change the model to 'resnet50' and the weights to weights="IMAGENET1K_V1" when needed.
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights=None).to(device)

# Background: The original resnet18 is designed for ImageNet dataset to predict 1000 classes.
# TODO: Change the output of the model to 10 class.
model.fc = nn.Linear(512, 10).to(device)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


### Training and Testing Models

In [ ]:
# TODO: Fill in the code cell according to the pytorch tutorial we gave.
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    epoch_loss = 0
    correct = 0

    model.train()

    for X, y in tqdm(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        pred = pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(y.view_as(pred)).sum().item()

    avg_epoch_loss = epoch_loss / num_batches
    avg_acc = correct / size

    return avg_epoch_loss, avg_acc

def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    epoch_loss = 0
    correct = 0

    model.eval()

    with torch.no_grad():
        for X, y in tqdm(dataloader):
            X, y = X.to(device), y.to(device)

            pred = model(X)

            epoch_loss += loss_fn(pred, y).item()
            pred = pred.argmax(dim=1, keepdim=True)
            correct += pred.eq(y.view_as(pred)).sum().item()

    avg_epoch_loss = epoch_loss / num_batches
    avg_acc = correct / size

    return avg_epoch_loss, avg_acc

### Resnet18

In [ ]:
##### Whole data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:01<00:00, 25.68it/s]


Epoch  1: Loss = 1.5322 Acc = 0.44 Test_Loss = 1.5080 Test_Acc = 0.44


100%|██████████| 40/40 [00:01<00:00, 25.50it/s]


Epoch  2: Loss = 1.2051 Acc = 0.57 Test_Loss = 1.1270 Test_Acc = 0.60


100%|██████████| 40/40 [00:01<00:00, 26.05it/s]


...

Epoch 18: Loss = 0.4919 Acc = 0.83 Test_Loss = 0.6371 Test_Acc = 0.79


100%|██████████| 40/40 [00:01<00:00, 25.72it/s]


Epoch 19: Loss = 0.4812 Acc = 0.83 Test_Loss = 0.6012 Test_Acc = 0.79


100%|██████████| 40/40 [00:01<00:00, 26.34it/s]

Epoch 20: Loss = 0.4621 Acc = 0.84 Test_Loss = 0.7490 Test_Acc = 0.76
Done!


In [ ]:
##### 1/2 data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(half_train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:01<00:00, 25.41it/s]


Epoch  1: Loss = 1.6547 Acc = 0.20 Test_Loss = 1.5232 Test_Acc = 0.46


100%|██████████| 40/40 [00:01<00:00, 25.16it/s]


Epoch  2: Loss = 1.3555 Acc = 0.26 Test_Loss = 1.4808 Test_Acc = 0.47


100%|██████████| 40/40 [00:01<00:00, 23.97it/s]


...

Epoch 18: Loss = 0.6586 Acc = 0.38 Test_Loss = 0.7972 Test_Acc = 0.71


100%|██████████| 40/40 [00:01<00:00, 23.57it/s]


Epoch 19: Loss = 0.6519 Acc = 0.38 Test_Loss = 0.8219 Test_Acc = 0.73


100%|██████████| 40/40 [00:02<00:00, 19.92it/s]

Epoch 20: Loss = 0.6229 Acc = 0.39 Test_Loss = 0.7485 Test_Acc = 0.74
Done!


In [ ]:
##### 1/16 data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(sixteenth_train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:01<00:00, 24.50it/s]


Epoch  1: Loss = 2.1224 Acc = 0.02 Test_Loss = 2.5166 Test_Acc = 0.11


100%|██████████| 40/40 [00:01<00:00, 24.09it/s]


Epoch  2: Loss = 1.8352 Acc = 0.02 Test_Loss = 3.5050 Test_Acc = 0.18


100%|██████████| 40/40 [00:01<00:00, 24.05it/s]


...

Epoch 18: Loss = 1.2841 Acc = 0.03 Test_Loss = 1.6975 Test_Acc = 0.42


100%|██████████| 40/40 [00:01<00:00, 25.14it/s]


Epoch 19: Loss = 1.2735 Acc = 0.03 Test_Loss = 1.2583 Test_Acc = 0.54


100%|██████████| 40/40 [00:01<00:00, 24.87it/s]

Epoch 20: Loss = 1.2348 Acc = 0.03 Test_Loss = 1.2784 Test_Acc = 0.54
Done!


### Resnet50

In [ ]:
# HINT: Remember to change the model to 'resnet50' and the weights to weights="IMAGENET1K_V1" when needed.
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=None).to(device)

# Background: The original resnet18 is designed for ImageNet dataset to predict 1000 classes.
# TODO: Change the output of the model to 10 class.
model.fc = nn.Linear(2048, 10).to(device)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
# TODO: Fill in the code cell according to the pytorch tutorial we gave.
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    epoch_loss = 0
    correct = 0

    model.train()

    for X, y in tqdm(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        pred = pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(y.view_as(pred)).sum().item()

    avg_epoch_loss = epoch_loss / num_batches
    avg_acc = correct / size

    return avg_epoch_loss, avg_acc

def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    epoch_loss = 0
    correct = 0

    model.eval()

    with torch.no_grad():
        for X, y in tqdm(dataloader):
            X, y = X.to(device), y.to(device)

            pred = model(X)

            epoch_loss += loss_fn(pred, y).item()
            pred = pred.argmax(dim=1, keepdim=True)
            correct += pred.eq(y.view_as(pred)).sum().item()

    avg_epoch_loss = epoch_loss / num_batches
    avg_acc = correct / size

    return avg_epoch_loss, avg_acc

In [ ]:
##### Whole data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:02<00:00, 18.86it/s]


Epoch  1: Loss = 1.8541 Acc = 0.35 Test_Loss = 1.5705 Test_Acc = 0.43


100%|██████████| 40/40 [00:02<00:00, 18.58it/s]


Epoch  2: Loss = 1.5411 Acc = 0.45 Test_Loss = 1.4130 Test_Acc = 0.50


100%|██████████| 40/40 [00:02<00:00, 18.73it/s]


...

Epoch 18: Loss = 0.7918 Acc = 0.72 Test_Loss = 0.7899 Test_Acc = 0.72


100%|██████████| 40/40 [00:02<00:00, 18.90it/s]


Epoch 19: Loss = 0.8359 Acc = 0.71 Test_Loss = 1.3548 Test_Acc = 0.60


100%|██████████| 40/40 [00:02<00:00, 19.05it/s]

Epoch 20: Loss = 0.8149 Acc = 0.72 Test_Loss = 0.8034 Test_Acc = 0.74
Done!


In [ ]:
##### 1/2 data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(half_train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:02<00:00, 19.11it/s]


Epoch  1: Loss = 2.0168 Acc = 0.15 Test_Loss = 1.7020 Test_Acc = 0.40


100%|██████████| 40/40 [00:02<00:00, 19.00it/s]


Epoch  2: Loss = 1.6534 Acc = 0.20 Test_Loss = 3.4878 Test_Acc = 0.46


100%|██████████| 40/40 [00:02<00:00, 18.86it/s]


...

Epoch 18: Loss = 1.3330 Acc = 0.26 Test_Loss = 1.1221 Test_Acc = 0.60


100%|██████████| 40/40 [00:02<00:00, 19.21it/s]


Epoch 19: Loss = 1.3697 Acc = 0.27 Test_Loss = 1.2774 Test_Acc = 0.54


100%|██████████| 40/40 [00:02<00:00, 18.79it/s]

Epoch 20: Loss = 1.4942 Acc = 0.24 Test_Loss = 2.0357 Test_Acc = 0.43
Done!


In [ ]:
##### 1/16 data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(sixteenth_train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:02<00:00, 18.86it/s]


Epoch  1: Loss = 2.8272 Acc = 0.01 Test_Loss = 2.5528 Test_Acc = 0.10


100%|██████████| 40/40 [00:02<00:00, 18.88it/s]


Epoch  2: Loss = 2.2485 Acc = 0.01 Test_Loss = 2.4539 Test_Acc = 0.15


100%|██████████| 40/40 [00:02<00:00, 18.32it/s]


...

Epoch 18: Loss = 1.7021 Acc = 0.02 Test_Loss = 1.9246 Test_Acc = 0.36


100%|██████████| 40/40 [00:02<00:00, 18.67it/s]


Epoch 19: Loss = 1.6305 Acc = 0.03 Test_Loss = 1.8191 Test_Acc = 0.39


100%|██████████| 40/40 [00:02<00:00, 18.63it/s]

Epoch 20: Loss = 1.6576 Acc = 0.03 Test_Loss = 1.5748 Test_Acc = 0.42
Done!


### Resnet18 (pretrained)

In [ ]:
# HINT: Remember to change the model to 'resnet50' and the weights to weights="IMAGENET1K_V1" when needed.
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights="IMAGENET1K_V1").to(device)

# Background: The original resnet18 is designed for ImageNet dataset to predict 1000 classes.
# TODO: Change the output of the model to 10 class.
model.fc = nn.Linear(512, 10).to(device)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
# TODO: Fill in the code cell according to the pytorch tutorial we gave.
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    epoch_loss = 0
    correct = 0

    model.train()

    for X, y in tqdm(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        pred = pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(y.view_as(pred)).sum().item()

    avg_epoch_loss = epoch_loss / num_batches
    avg_acc = correct / size

    return avg_epoch_loss, avg_acc

def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    epoch_loss = 0
    correct = 0

    model.eval()

    with torch.no_grad():
        for X, y in tqdm(dataloader):
            X, y = X.to(device), y.to(device)

            pred = model(X)

            epoch_loss += loss_fn(pred, y).item()
            pred = pred.argmax(dim=1, keepdim=True)
            correct += pred.eq(y.view_as(pred)).sum().item()

    avg_epoch_loss = epoch_loss / num_batches
    avg_acc = correct / size

    return avg_epoch_loss, avg_acc

In [ ]:
##### Whole data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:01<00:00, 24.09it/s]


Epoch  1: Loss = 0.9690 Acc = 0.67 Test_Loss = 0.7260 Test_Acc = 0.75


100%|██████████| 40/40 [00:01<00:00, 24.22it/s]


Epoch  2: Loss = 0.6864 Acc = 0.76 Test_Loss = 0.7562 Test_Acc = 0.75


100%|██████████| 40/40 [00:01<00:00, 23.81it/s]


...

Epoch 18: Loss = 0.2911 Acc = 0.90 Test_Loss = 0.6545 Test_Acc = 0.80


100%|██████████| 40/40 [00:01<00:00, 21.83it/s]


Epoch 19: Loss = 0.2915 Acc = 0.90 Test_Loss = 0.4694 Test_Acc = 0.85


100%|██████████| 40/40 [00:01<00:00, 25.53it/s]

Epoch 20: Loss = 0.2812 Acc = 0.90 Test_Loss = 0.4489 Test_Acc = 0.85
Done!


In [ ]:
##### 1/2 data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(half_train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:01<00:00, 24.67it/s]


Epoch  1: Loss = 1.1067 Acc = 0.31 Test_Loss = 0.8146 Test_Acc = 0.72


100%|██████████| 40/40 [00:01<00:00, 21.85it/s]


Epoch  2: Loss = 0.7919 Acc = 0.36 Test_Loss = 0.7873 Test_Acc = 0.74


100%|██████████| 40/40 [00:01<00:00, 24.98it/s]


...

Epoch 18: Loss = 0.3807 Acc = 0.43 Test_Loss = 0.5576 Test_Acc = 0.82


100%|██████████| 40/40 [00:01<00:00, 24.83it/s]


Epoch 19: Loss = 0.3885 Acc = 0.43 Test_Loss = 0.5247 Test_Acc = 0.83


100%|██████████| 40/40 [00:01<00:00, 25.68it/s]

Epoch 20: Loss = 0.3735 Acc = 0.44 Test_Loss = 0.5058 Test_Acc = 0.84
Done!


In [ ]:
##### 1/16 data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(sixteenth_train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:01<00:00, 24.30it/s]


Epoch  1: Loss = 1.7601 Acc = 0.02 Test_Loss = 2.2021 Test_Acc = 0.28


100%|██████████| 40/40 [00:01<00:00, 23.64it/s]


Epoch  2: Loss = 1.3347 Acc = 0.03 Test_Loss = 1.5237 Test_Acc = 0.54


100%|██████████| 40/40 [00:01<00:00, 25.06it/s]


...

Epoch 18: Loss = 0.7985 Acc = 0.05 Test_Loss = 0.9335 Test_Acc = 0.68


100%|██████████| 40/40 [00:01<00:00, 23.44it/s]


Epoch 19: Loss = 0.7646 Acc = 0.05 Test_Loss = 1.1139 Test_Acc = 0.64


100%|██████████| 40/40 [00:01<00:00, 24.37it/s]

Epoch 20: Loss = 0.7511 Acc = 0.05 Test_Loss = 0.7823 Test_Acc = 0.74
Done!


### Resnet50 (pretrained)

In [ ]:
# HINT: Remember to change the model to 'resnet50' and the weights to weights="IMAGENET1K_V1" when needed.
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights="IMAGENET1K_V1").to(device)

# Background: The original resnet18 is designed for ImageNet dataset to predict 1000 classes.
# TODO: Change the output of the model to 10 class.
model.fc = nn.Linear(2048, 10).to(device)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
# TODO: Fill in the code cell according to the pytorch tutorial we gave.
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    epoch_loss = 0
    correct = 0

    model.train()

    for X, y in tqdm(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        pred = pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(y.view_as(pred)).sum().item()

    avg_epoch_loss = epoch_loss / num_batches
    avg_acc = correct / size

    return avg_epoch_loss, avg_acc

def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    epoch_loss = 0
    correct = 0

    model.eval()

    with torch.no_grad():
        for X, y in tqdm(dataloader):
            X, y = X.to(device), y.to(device)

            pred = model(X)

            epoch_loss += loss_fn(pred, y).item()
            pred = pred.argmax(dim=1, keepdim=True)
            correct += pred.eq(y.view_as(pred)).sum().item()

    avg_epoch_loss = epoch_loss / num_batches
    avg_acc = correct / size

    return avg_epoch_loss, avg_acc

In [ ]:
##### Whole data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:02<00:00, 19.04it/s]


Epoch  1: Loss = 0.9368 Acc = 0.68 Test_Loss = 0.6595 Test_Acc = 0.77


100%|██████████| 40/40 [00:02<00:00, 19.22it/s]


Epoch  2: Loss = 0.6637 Acc = 0.77 Test_Loss = 0.8044 Test_Acc = 0.73


100%|██████████| 40/40 [00:02<00:00, 18.95it/s]


...

Epoch 18: Loss = 0.3495 Acc = 0.88 Test_Loss = 0.4538 Test_Acc = 0.85


100%|██████████| 40/40 [00:02<00:00, 18.76it/s]


Epoch 19: Loss = 0.2916 Acc = 0.90 Test_Loss = 0.5755 Test_Acc = 0.82


100%|██████████| 40/40 [00:02<00:00, 18.91it/s]

Epoch 20: Loss = 0.2734 Acc = 0.90 Test_Loss = 0.4041 Test_Acc = 0.87
Done!


In [ ]:
##### 1/2 data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(half_train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:02<00:00, 18.83it/s]


Epoch  1: Loss = 1.0960 Acc = 0.31 Test_Loss = 1.0276 Test_Acc = 0.69


100%|██████████| 40/40 [00:02<00:00, 14.45it/s]


Epoch  2: Loss = 0.7567 Acc = 0.37 Test_Loss = 0.7083 Test_Acc = 0.76


100%|██████████| 40/40 [00:02<00:00, 19.10it/s]


...

Epoch 18: Loss = 0.3753 Acc = 0.44 Test_Loss = 0.4514 Test_Acc = 0.85


100%|██████████| 40/40 [00:02<00:00, 19.10it/s]


Epoch 19: Loss = 0.3597 Acc = 0.44 Test_Loss = 0.4729 Test_Acc = 0.84


100%|██████████| 40/40 [00:02<00:00, 18.88it/s]

Epoch 20: Loss = 0.3427 Acc = 0.44 Test_Loss = 0.4742 Test_Acc = 0.84
Done!


In [ ]:
##### 1/16 data #####
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train(sixteenth_train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
print("Done!")

100%|██████████| 40/40 [00:02<00:00, 19.03it/s]


Epoch  1: Loss = 1.8485 Acc = 0.02 Test_Loss = 15.7038 Test_Acc = 0.17


100%|██████████| 40/40 [00:02<00:00, 18.69it/s]


Epoch  2: Loss = 1.3255 Acc = 0.03 Test_Loss = 1.9397 Test_Acc = 0.51


100%|██████████| 40/40 [00:02<00:00, 19.05it/s]


...

Epoch 18: Loss = 0.6640 Acc = 0.05 Test_Loss = 0.7548 Test_Acc = 0.75


100%|██████████| 40/40 [00:02<00:00, 18.89it/s]


Epoch 19: Loss = 0.6871 Acc = 0.05 Test_Loss = 0.7418 Test_Acc = 0.75


100%|██████████| 40/40 [00:02<00:00, 19.10it/s]

Epoch 20: Loss = 0.7168 Acc = 0.05 Test_Loss = 0.7994 Test_Acc = 0.74
Done!


## Codes for Problem 3

In [ ]:
# TODO: Try to achieve the best performance given all training data using whatever model and training strategy.
batch_size = 64

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    # transforms.RandomRotation(10),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=train_transform)
valid_dataset = datasets.CIFAR10(root='data', train=False, download=True, transform=test_transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [ ]:
# HINT: Remember to change the model to 'resnet50' and the weights to weights="IMAGENET1K_V1" when needed.
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', weights="IMAGENET1K_V2").to(device)

# Background: The original resnet18 is designed for ImageNet dataset to predict 1000 classes.
# TODO: Change the output of the model to 10 class.
model.fc = nn.Linear(2048, 10).to(device)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model = torch.load("./gdrive/MyDrive/cv_hw3_224.pt")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    epoch_loss = 0
    correct = 0

    model.train()

    for X, y in tqdm(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        pred = pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(y.view_as(pred)).sum().item()

    avg_epoch_loss = epoch_loss / num_batches
    avg_acc = correct / size

    return avg_epoch_loss, avg_acc

def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    epoch_loss = 0
    correct = 0

    model.eval()

    with torch.no_grad():
        for X, y in tqdm(dataloader):
            X, y = X.to(device), y.to(device)

            pred = model(X)

            epoch_loss += loss_fn(pred, y).item()
            pred = pred.argmax(dim=1, keepdim=True)
            correct += pred.eq(y.view_as(pred)).sum().item()

    avg_epoch_loss = epoch_loss / num_batches
    avg_acc = correct / size

    return avg_epoch_loss, avg_acc

In [ ]:
import sys
epochs = 400
best_loss = sys.maxsize
previous_loss = sys.maxsize
patience = 0
for epoch in range(epochs):
    train_loss, train_acc = train(train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
    if best_loss > test_loss:
      best_loss = test_loss
      torch.save(model, "./gdrive/MyDrive/cv_hw3_224.pt")
    if previous_loss < test_loss:
      patience += 1
      if patience > 2:
        print("Early Stopping")
        break
    else:
      patience = 0
    previous_loss = test_loss
print("Done!")

100%|██████████| 157/157 [01:24<00:00,  1.86it/s]


Epoch  1: Loss = 0.5138 Acc = 0.83 Test_Loss = 0.4199 Test_Acc = 0.86


100%|██████████| 157/157 [01:24<00:00,  1.87it/s]


Epoch  2: Loss = 0.3057 Acc = 0.89 Test_Loss = 0.3914 Test_Acc = 0.87


100%|██████████| 157/157 [01:24<00:00,  1.87it/s]


...

100%|██████████| 157/157 [01:23<00:00,  1.88it/s]


Epoch  9: Loss = 0.1141 Acc = 0.96 Test_Loss = 0.2385 Test_Acc = 0.93


100%|██████████| 782/782 [20:24<00:00,  1.57s/it]

100%|██████████| 157/157 [01:23<00:00,  1.88it/s]



Epoch 10: Loss = 0.1030 Acc = 0.96 Test_Loss = 0.2038 Test_Acc = 0.94
Epoch 10: Loss = 0.1030 Acc = 0.96 Test_Loss = 0.2038 Test_Acc = 0.94


 46%|████▌     | 358/782 [09:20<11:01,  1.56s/it]

In [ ]:
import sys
epochs = 400
best_loss = sys.maxsize
previous_loss = sys.maxsize
patience = 0
for epoch in range(11, epochs):
    train_loss, train_acc = train(train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
    if best_loss > test_loss:
      best_loss = test_loss
      torch.save(model, "./gdrive/MyDrive/cv_hw3_224.pt")
    if previous_loss < test_loss:
      patience += 1
      if patience > 2:
        print("Early Stopping")
        break
    else:
      patience = 0
    previous_loss = test_loss
print("Done!")

100%|██████████| 157/157 [01:21<00:00,  1.92it/s]


Epoch 12: Loss = 0.1039 Acc = 0.96 Test_Loss = 0.2047 Test_Acc = 0.94


100%|██████████| 157/157 [01:21<00:00,  1.92it/s]


Epoch 13: Loss = 0.0881 Acc = 0.97 Test_Loss = 0.2130 Test_Acc = 0.94


100%|██████████| 157/157 [01:21<00:00,  1.92it/s]


...

100%|██████████| 157/157 [01:21<00:00,  1.91it/s]


Epoch 15: Loss = 0.0709 Acc = 0.98 Test_Loss = 0.2011 Test_Acc = 0.94


100%|██████████| 157/157 [01:22<00:00,  1.89it/s]


Epoch 16: Loss = 0.0662 Acc = 0.98 Test_Loss = 0.2157 Test_Acc = 0.94


  3%|▎         | 26/782 [00:39<19:10,  1.52s/it]

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
import sys
epochs = 400
best_loss = sys.maxsize
previous_loss = sys.maxsize
patience = 0
for epoch in range(16, epochs):
    train_loss, train_acc = train(train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, model, loss_fn)
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f} Acc = {train_acc:.2f} Test_Loss = {test_loss:.4f} Test_Acc = {test_acc:.2f}")
    if best_loss > test_loss:
      best_loss = test_loss
      torch.save(model, "./gdrive/MyDrive/cv_hw3_224.pt")
    if previous_loss < test_loss:
      patience += 1
      if patience > 2:
        print("Early Stopping")
        break
    else:
      patience = 0
    previous_loss = test_loss
print("Done!")

100%|██████████| 157/157 [01:23<00:00,  1.89it/s]


Epoch 17: Loss = 0.0194 Acc = 0.99 Test_Loss = 0.1355 Test_Acc = 0.96


100%|██████████| 157/157 [01:23<00:00,  1.89it/s]


Epoch 18: Loss = 0.0065 Acc = 1.00 Test_Loss = 0.1469 Test_Acc = 0.96


100%|██████████| 157/157 [01:23<00:00,  1.89it/s]


...

100%|██████████| 157/157 [01:22<00:00,  1.89it/s]


Epoch 21: Loss = 0.0068 Acc = 1.00 Test_Loss = 0.1567 Test_Acc = 0.97


100%|██████████| 157/157 [01:25<00:00,  1.84it/s]


Epoch 22: Loss = 0.0062 Acc = 1.00 Test_Loss = 0.1541 Test_Acc = 0.97


  3%|▎         | 27/782 [00:42<19:46,  1.57s/it]

## Problems

1. (30%) Finish the rest of the codes for Problem 1 and Problem 2 according to the hint. (2 code cells in total.)
2. Train small model (resnet18) and big model (resnet50) from scratch on `sixteenth_train_dataloader`, `half_train_dataloader`, and `train_dataloader` respectively.
3. (30%) Achieve the best performance given all training data using whatever model and training strategy.  
  (You cannot use the model that was pretrained on CIFAR10)



## Discussion


- (30%) The relationship between the accuracy, model size, and the training dataset size.  
    (Total 6 models. Small model trains on the sixteenth, half, and all data. Big model trains on the sixteenth, half, and all data.)
- (10%) What if we train the ResNet with ImageNet initialized weights (`weights="IMAGENET1K_V1"`), how would the relationship change?

## Credits

1. [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)